In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
pip install pyproj

In [ ]:
from pyproj import Geod

In [ ]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/lat-long-data.csv')

In [ ]:
data

,sourceid,dstid,hod,mean_travel_time,S_geometry,D_geometry,Source_Longitude,Source_Latitude,dst_Longitude,dst_Latitude
0,1,1,23,233.60,POINT (77.13179986460109 28.621414737282),POINT (77.13179986460109 28.621414737282),77.131800,28.621415,77.131800,28.621415
1,1,2,0,201.43,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.131800,28.621415,77.142895,28.610587
2,1,2,1,179.93,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.131800,28.621415,77.142895,28.610587
3,1,2,2,165.50,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.131800,28.621415,77.142895,28.610587
4,1,2,3,158.29,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.131800,28.621415,77.142895,28.610587
...,...,...,...,...,...,...,...,...,...,...
1455927,290,279,16,2281.00,POINT (77.08282193508637 28.57316948198418),POINT (77.06904180662636 28.63870326950842),77.082822,28.573169,77.069042,28.638703
1455928,290,279,17,2848.11,POINT (77.08282193508637 28.57316948198418),POINT (77.06904180662636 28.63870326950842),77.082822,28.573169,77.069042,28.638703
1455929,290,281,15,1937.00,POINT (77.08282193508637 28.57316948198418),POINT (77.07672407631429 28.63298613132645),77.082822,28.573169,77.076724,28.632986
1455930,290,281,16,2190.38,POINT (77.08282193508637 28.57316948198418),POINT (77.07672407631429 28.63298613132645),77.082822,28.573169,77.076724,28.632986


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455932 entries, 0 to 1455931
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sourceid          1455932 non-null  int64  
 1   dstid             1455932 non-null  int64  
 2   hod               1455932 non-null  int64  
 3   mean_travel_time  1455932 non-null  float64
 4   S_geometry        1455932 non-null  object 
 5   D_geometry        1455932 non-null  object 
 6   Source_Longitude  1455932 non-null  float64
 7   Source_Latitude   1455932 non-null  float64
 8   dst_Longitude     1455932 non-null  float64
 9   dst_Latitude      1455932 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 111.1+ MB


In [ ]:
wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method

#Get distance between pairs of lat-lon points
def Distance(lat1,lon1,lat2,lon2):
  az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
  return dist

#Create test data
lat1 = np.random.uniform(-90,90,100)
lon1 = np.random.uniform(-180,180,100)
lat2 = np.random.uniform(-90,90,100)
lon2 = np.random.uniform(-180,180,100)

#Add/update a column to the data frame with the distances (in metres)
data['dist'] = Distance(data['Source_Latitude'].tolist(),data['Source_Longitude'].tolist(),data['dst_Latitude'].tolist(),data['dst_Longitude'].tolist())

In [ ]:
data['distance in Km']=data['dist']*1000

In [ ]:
data.drop(['dist'], axis=1, inplace=True)

In [ ]:
data.head()

,sourceid,dstid,hod,mean_travel_time,S_geometry,D_geometry,Source_Longitude,Source_Latitude,dst_Longitude,dst_Latitude,distance in Km
0,1,1,8,233.60,POINT (77.13179986460109 28.621414737282),POINT (77.13179986460109 28.621414737282),77.1318,28.621415,77.131800,28.621415,0.000000e+00
1,1,2,6,201.43,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.1318,28.621415,77.142895,28.610587,1.617816e+06
2,1,2,4,179.93,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.1318,28.621415,77.142895,28.610587,1.617816e+06
3,1,2,1,165.50,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.1318,28.621415,77.142895,28.610587,1.617816e+06
4,1,2,0,158.29,POINT (77.13179986460109 28.621414737282),POINT (77.14289455633259 28.61058711806038),77.1318,28.621415,77.142895,28.610587,1.617816e+06


In [ ]:
labels_ordered=data.groupby(['hod'])['mean_travel_time'].mean().sort_values().index
labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
data['hod']=data['hod'].map(labels_ordered)

In [ ]:
data['sourceid']=data['sourceid'].astype(str)
data['dstid']=data['dstid'].astype(str)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1455932 entries, 0 to 1455931
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sourceid          1455932 non-null  object 
 1   dstid             1455932 non-null  object 
 2   hod               1455932 non-null  int64  
 3   mean_travel_time  1455932 non-null  float64
 4   S_geometry        1455932 non-null  object 
 5   D_geometry        1455932 non-null  object 
 6   Source_Longitude  1455932 non-null  float64
 7   Source_Latitude   1455932 non-null  float64
 8   dst_Longitude     1455932 non-null  float64
 9   dst_Latitude      1455932 non-null  float64
 10  distance in Km    1455932 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 122.2+ MB


In [ ]:
if 0 in data['distance in Km'].unique():
    pass
else:
    data['distance in Km']=np.log(data['distance in Km'])

In [ ]:
x=data.iloc[:,[0,1,2,10]]
y=data.iloc[:,3]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor 
regressor = RandomForestRegressor()
regressor.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred=regressor.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)

In [ ]:
score

0.9529555244619259